# Desiging recommanding engine that match users with eachother 

In [1]:
# import libraries 
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


# set display rows to 1000
pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

In [2]:
data = pd.read_csv('Community.csv')

In [3]:
data.head()

,First Name,Last Name,Gender,Age,Interest
0,Jenna,Murray,Female,18,p
1,Lyndon,Crawford,Male,22,g
2,Vanessa,Hunt,Female,25,e
3,Ted,Barrett,Male,22,l
4,Edwin,Harris,Male,18,j


In [4]:
data= data.reset_index().rename({'index':'user_id'}, axis =1) 

In [5]:
data.head()

,user_id,First Name,Last Name,Gender,Age,Interest
0,0,Jenna,Murray,Female,18,p
1,1,Lyndon,Crawford,Male,22,g
2,2,Vanessa,Hunt,Female,25,e
3,3,Ted,Barrett,Male,22,l
4,4,Edwin,Harris,Male,18,j


In [6]:
data.Interest.value_counts()

y    13
c    11
j    11
w    11
t    11
a    10
b    10
x     9
g     9
p     8
d     8
h     8
i     7
z     7
l     7
k     7
u     6
v     6
s     6
q     6
e     6
m     6
f     5
o     4
r     4
n     4
Name: Interest, dtype: int64

In [7]:
interests_dict = {'y': 'Devoloper',
'w': 'Data Science',
'j': 'basketball',
'c': 'Games',
't': 'Hicking',
'b': 'Music',
'a': 'Coffee',
'g': 'Entrepreneurs',
'p': 'Vollyball',
'h': 'Football',
'd': 'Anime',
'z': 'Moveies',
'l': 'Gym',
'i': 'Diet',
'k': 'Self development',
'v': 'History',
'u': 'Religon',
's': 'Travel',
'e': 'Diving',
'q': 'Swimming',
'm': 'Fishing',
'f': 'Hunting',
'o': 'Cars',
'n': 'Techenology',
'r': 'Food',
'x': 'Phtoghraphy'}
interests_list = list(interests_dict.values())

In [8]:
data.Interest = data.Interest.map(interests_dict)

In [9]:
import random

data['Interest2']= ''
data['Interest3']= ''
data['Interest4']= ''
data['Interest5']= ''

for i in range(200):
    random.seed(i)
    data.loc[i,['Interest','Interest2','Interest3','Interest4','Interest5']] = random.sample(interests_list, 5)

In [10]:
cols = ['Interest','Interest2','Interest3','Interest4','Interest5']
data['combined'] = data[cols].apply(lambda row: ', '.join(row.values.astype(str)), axis=1)

In [11]:
data.sample(10)

,user_id,First Name,Last Name,Gender,Age,Interest,Interest2,Interest3,Interest4,Interest5,combined
54,54,Nicholas,Moore,Male,29,Hicking,Self development,Travel,Football,History,"Hicking, Self development, Travel, Football, H..."
124,124,Kate,Elliott,Female,28,Vollyball,Travel,Devoloper,Music,Data Science,"Vollyball, Travel, Devoloper, Music, Data Science"
111,111,Honey,Bennett,Female,18,Coffee,Anime,History,Gym,Diet,"Coffee, Anime, History, Gym, Diet"
115,115,Frederick,Payne,Male,24,Football,Coffee,Food,Games,Travel,"Football, Coffee, Food, Games, Travel"
184,184,Hailey,Henderson,Female,28,Vollyball,Devoloper,Entrepreneurs,Cars,Anime,"Vollyball, Devoloper, Entrepreneurs, Cars, Anime"
84,84,Valeria,Ross,Female,27,Techenology,Football,Food,Data Science,History,"Techenology, Football, Food, Data Science, His..."
11,11,Rebecca,Johnson,Female,20,Self development,Travel,Food,Religon,Diving,"Self development, Travel, Food, Religon, Diving"
75,75,Isabella,Murphy,Female,30,Self development,Diving,Diet,Data Science,Techenology,"Self development, Diving, Diet, Data Science, ..."
81,81,Eddy,Elliott,Male,20,Religon,Self development,Anime,Travel,Gym,"Religon, Self development, Anime, Travel, Gym"
19,19,Samantha,Payne,Female,22,Hunting,Data Science,Phtoghraphy,Religon,Games,"Hunting, Data Science, Phtoghraphy, Religon, G..."


In [12]:
# implement recommanding system (C)

In [13]:
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
community_tfidf = TfidfVectorizer()

In [15]:
community_description_matrix = community_tfidf.fit_transform(data['combined'])

In [16]:
community_description_matrix.shape

(200, 28)

In [17]:
community_description_matrix.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.41691144],
       [0.        , 0.4489234 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.40555306, 0.        , ..., 0.40177937, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.45955016, ..., 0.        , 0.43350401,
        0.        ],
       [0.        , 0.        , 0.42855713, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.42411332, 0.        ,
        0.44506606]])

In [18]:
cosine_similarity = linear_kernel(community_description_matrix, community_description_matrix)

In [19]:
indices = pd.Series(data[['Interest','Interest2','Interest3','Interest4','Interest5']].index)

In [20]:
def recommend(index, cosine_sim=cosine_similarity):
    id = indices[index]
    
    # Get the pairwsie similarity scores of all books compared to that book, 
    # sorting them and getting top 5
    
    similarity_scores = list(enumerate(cosine_sim[id]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similarity_scores = similarity_scores[1:6]

    # Get the books index
    community_index = [i[0] for i in similarity_scores]

    # Return the top 5 most similar books using integer-location based indexing (iloc)
    return data.iloc[community_index]

In [21]:
recommend(10)

,user_id,First Name,Last Name,Gender,Age,Interest,Interest2,Interest3,Interest4,Interest5,combined
136,136,Alford,Montgomery,Male,21,Diving,Diet,Religon,Data Science,History,"Diving, Diet, Religon, Data Science, History"
98,98,Paige,Parker,Female,27,Moveies,Diving,Data Science,Diet,History,"Moveies, Diving, Data Science, Diet, History"
165,165,Dominik,Wells,Male,19,Devoloper,Data Science,Cars,Diet,basketball,"Devoloper, Data Science, Cars, Diet, basketball"
127,127,Mike,Hawkins,Male,24,Data Science,Swimming,Diving,Games,Devoloper,"Data Science, Swimming, Diving, Games, Devoloper"
198,198,Lucas,Carroll,Male,29,Data Science,Diving,Cars,History,Games,"Data Science, Diving, Cars, History, Games"


# Desiging recommanding engine that match users with community 

In [73]:
df= pd.read_csv('community-2.csv')

In [40]:
df= df.reset_index().rename({'index':'user_id'}, axis =1) 

In [41]:
df.head()

,user_id,community_id,community_ratings
0,0,241,1
1,1,226,0
2,2,230,3
3,3,237,5
4,4,290,2


In [74]:
df.community_id.value_counts()

230    5
371    4
398    4
322    4
306    3
375    3
343    3
228    3
229    3
324    3
313    3
256    3
304    3
241    3
267    3
308    2
352    2
391    2
390    2
389    2
376    2
258    2
367    2
363    2
354    2
273    2
351    2
311    2
342    2
329    2
327    2
279    2
284    2
286    2
325    2
301    2
393    2
255    2
358    2
248    2
237    2
238    2
220    2
246    2
216    2
214    2
217    2
203    2
207    2
262    1
240    1
290    1
291    1
293    1
295    1
297    1
299    1
300    1
302    1
289    1
303    1
252    1
257    1
236    1
309    1
235    1
234    1
259    1
288    1
263    1
287    1
265    1
251    1
269    1
272    1
274    1
275    1
277    1
261    1
278    1
260    1
247    1
280    1
233    1
283    1
243    1
315    1
200    1
232    1
378    1
365    1
366    1
370    1
373    1
374    1
212    1
211    1
377    1
381    1
359    1
385    1
387    1
208    1
205    1
392    1
394    1
396    1
201    1
361    1
254    1
231    1
2

In [42]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap ={}
ptr=0;
testdf= df

testdf=testdf[['user_id','community_ratings']].groupby(testdf['community_id'])
for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict)

In [79]:
len(reverseIndexMap)

131

In [43]:
from sklearn.feature_extraction import DictVectorizer

dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)

In [44]:
from sklearn.metrics.pairwise import cosine_similarity

pairwiseSimilarity = cosine_similarity(vector)

In [80]:
def printCommunityDetails(user_id):
    print("community:", data[data['user_id']== user_id]['combined'].values[0])
    print("First Name:",data[data['First Name']==user_id])
    print("=================++++++++++++++=========================")


def getTopRecommandations(community_id):
    row = reverseIndexMap[community_id]
    print("------INPUT BOOK--------")
    printCommunityDetails(community_id)
    print("-------RECOMMENDATIONS----------")
    similarBookIDs = [printCommunityDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]]

In [82]:
getTopRecommandations(200)

------INPUT BOOK--------


IndexError: index 0 is out of bounds for axis 0 with size 0